# Assignment 3 on Natural Language Processing

## Date : 30th Sept, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.



Please submit with outputs. 

In [1]:
import re
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jalend-15/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Load the IMDB dataset. You can load it using pandas as dataframe
import pandas as pd
import csv
ddf=pd.read_csv("IMDB Dataset.csv")
df=pd.read_csv("IMDB Dataset.csv")
rows= ddf.review
print(type(rows))


<class 'pandas.core.series.Series'>


# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [3]:
#imports
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
stopwords=stopwords.words("english")
from nltk.stem import SnowballStemmer
ss=SnowballStemmer("english")
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet') # Since Lemmatization method is based on WorldNet's built-in morph function.
lemmatizer=WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/jalend-15/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
#removing html tags
def remove_html_tags(text):     
    clean = re.compile('<.*?>')    
    return re.sub(clean, '', text)
#removing urls
def remove_URLS(text):
    return re.sub(r'http\S+', '', text)

df['clean_web']=[remove_URLS(remove_html_tags(text)) for text in df['review']]

#remove non alpha numeric characters, stopwords, and then lemmatize and stemming
def preprocess(text):
    sentences=sent_tokenize(text)
    preprocessed_text=[];
    for sentence in sentences:
        tokenizer = nltk.RegexpTokenizer(r"[a-z0-9]+")
        tokenized_sentence = tokenizer.tokenize(sentence.lower())
        for word in tokenized_sentence:
            if word not in stopwords:
                lw=lemmatizer.lemmatize(word)
                slw=ss.stem(lw)
                preprocessed_text.append(slw)
    return preprocessed_text
            
df['preprocessed_review']=[preprocess(text) for text in df['clean_web']]
df['review_len']=[len(processed_list) for processed_list in df['preprocessed_review']]

df.drop(['review','clean_web'], axis=1, inplace= True)


In [5]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels

totalWords=0
for row in df['preprocessed_review']:
    totalWords= totalWords+ len(row)
totalSentences = len(df['preprocessed_review'])
print(totalWords)
avgLength= totalWords/totalSentences
print(avgLength)

print("count of each label:")
ddf['sentiment'].value_counts()



5979119
119.58238
count of each label:


positive    25000
negative    25000
Name: sentiment, dtype: int64

In [6]:
'''
5979119
119.58238
count of each label:

negative    25000
positive    25000
Name: sentiment, dtype: int64
'''

'\n5979119\n119.58238\ncount of each label:\n\nnegative    25000\npositive    25000\nName: sentiment, dtype: int64\n'

# Naive Bayes classifier

In [7]:
# get reviews column from df
df['preprocessed_review']=[" ".join(review) for review in df['preprocessed_review'].values]
reviews = df['preprocessed_review']
# get labels column from df
labels = df['sentiment']

In [8]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)
print((encoded_labels))


#for lab in encoded_labels:
#    print(lab)


# print(enc.classes_)

[1 1 1 ... 0 0 0]


In [66]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(reviews, encoded_labels,stratify=encoded_labels,test_size=0.20,)
# train_sentences, test_sentences, train_labels, test_labels
#train_sentences= X_train
#test_sentences = X_test
#train_labels = y_train
#test_labels = y_test

Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [67]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter, defaultdict





cv1 = CountVectorizer(max_features=3000)

X_train0 = X_train[y_train==0]
X_train1 = X_train[y_train==1]
#X_train0 contains all negative tag words
#X_train1 contains all positive tag words
cv_fit1=cv1.fit_transform(X_train)

word_list= cv1.get_feature_names()

#word_list contains all the vocabulary

'''
max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(Sentence_list)
'''

'\nmax_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.\nvec = CountVectorizer(max_features = 3000)\nX = vec.fit_transform(Sentence_list)\n'

In [68]:
# Use laplace smoothing for words in test set not present in vocab of train set

cv_pos=  CountVectorizer(max_features=3000)
pfreq=  cv_pos.fit_transform(X_train1.values.tolist())
pos= pfreq.toarray()
pos_list= cv_pos.get_feature_names()

count1_list1 = pfreq.toarray().sum(axis=0)

#count1_list1 contains all frequencies of positive tag words
cv_neg=  CountVectorizer(max_features=3000)
nfreq=  cv_neg.fit_transform(X_train0.values.tolist())
neg= nfreq.toarray()
neg_list= cv_neg.get_feature_names()
count0_list0 = nfreq.toarray().sum(axis=0)
#count0_list0 contains all frequencies of negative tag words

counts= dict()
N1=0
N2=0
prob = defaultdict(lambda: defaultdict(lambda: 0))
i=0
for word in neg_list:
    prob[word][0]+= count0_list0[i]
    i+=1


    
i=0
for word in pos_list:
    prob[word][1]+= count1_list1[i]
    i+=1


In [69]:
# Build the model. Don't use the model from sklearn
import math
pos = math.log10(len(X_train1)/(len(X_train)))
neg = math.log10(len(X_train0)/(len(X_train)))


V= len(word_list)
xx=0
xx1=0
for word in word_list:
    if prob[word][1]!=0:
        xx+=prob[word][1]
       # print(counts[word][1],prob[word][1])
    if prob[word][0]!=0:
        xx1+=prob[word][0]
N1= xx
N2=xx1

for word in word_list:
        #print(prob1[word])
        prob[word][1]= math.log10((prob[word][1]+1)/(N1+V))
        prob[word][0]= math.log10((prob[word][0]+1)/(N2+V))
 #       print(prob[word][0]
 #       print(prob[word][1])
 #       print('\n')

265 212
3173 3590
389 377
236 293
243 259
247 180
101 152
138 325
506 125
136 130
123 125
119 177
105 136
105 144
117 172
110 507
109 117
700 198
110 151
114 137
105 147
164 116
608 176
110 461
350 363
478 449
344 388
520 600
747 774
634 350
212 225
437 491
927 1175
114 1286
1681 145
404 205
291 313
156 309
154 367
737 724
547 146
252 302
191 193
149 199
150 267
283 224
237 220
240 371
155 179
324 573
781 798
8494 5521
2777 2987
181 237
5805 5144
1180 1227
4765 3230
652 652
279 308
579 697
938 974
164 245
334 433
129 169
285 428
670 652
98 154
117 257
511 773
193 232
101 168
375 790
193 147
291 139
191 134
320 385
205 474
114 261
267 161
118 245
202 156
128 155
145 1594
1017 374
401 920
710 658
124 332
274 257
168 192
309 757
118 145
124 266
159 107
221 263
227 162
189 257
95 148
103 168
171 213
95 254
174 498
113 140
867 407
398 260
231 894
107 2470
706 801
2511 1688
801 974
1161 8610
213 154
5726 207
125 2364
199 3259
1854 1985
304 159
317 135
724 675
469 2340
119 778
488 149
121 467

6928 392
532 160
95 8165
809 532
102 817
660 385
134 398
3406 581
429 4149
153 518
387 214
440 2801
115 114
1109 911
154 156
752 888
102 121
230 769
1672 428
469 145
172 619
682 171
99 194
1128 1871
381 123
232 602
114 120
191 164
122 1281
229 511
1201 154
291 362
152 110
776 210
276 1959
310 333
970 256
222 287
197 156
331 515
3452 305
164 110
168 145
620 399
226 331
103 381
272 1389
150 270
104 146
102 117
99 162
140 2411
545 148
201 117
154 727
2409 287
2199 118
269 208
268 192
1695 265
1995 196
225 145
309 244
185 162
105 632
300 253
238 157
668 2156
111 1542
616 193
227 328
164 1317
150 1729
199 327
334 354
151 180
304 194
216 310
142 328
412 462
645 373
123 179
132 105
224 153
226 283
95 166
99 359
528 270
732 216
112 476
106 1019
229 280
153 182
102 369
199 568
133 775
500 114
554 216
118 158
304 161
2673 111
125 212
121 629
1320 365
286 170
101 366
1004 2382
864 147
121 1119
2581 335
217 148
391 1141
114 725
147 375
175 188
192 3404
126 241
142 155
375 469
109 160
112 111
243 3

In [73]:
# Test the model on test set and report Accuracy
def predict(sentence):
    p1=pos
    n1=neg
    for word in sentence.split():
        p1+= prob[word][1]
        n1+= prob[word][0]
        
        #print(word)
    #print(p1)
    #print(n1)
#    print('\n')
    if p1>n1:
        return 1
    else :
        return 0


In [74]:
predictions = X_test.apply(lambda x:predict(x)).values

In [75]:
from sklearn.metrics import accuracy_score

print("Accuracy = ",accuracy_score(y_test,predictions))

truepos=0
trueneg=0
falseneg=0
falsepos=0
i=0

for sentence in X_test:
    prd= predict(sentence)
    #print(prd, "+" ,y_test[i])
    if prd == y_test[i]:
        if prd == 1:
            truepos+=1
        else:
            trueneg+=1
    else:
        if prd==1:
            falsepos+=1
        else:
            falseneg+=1
    i+=1
#print(trueneg)
#print(truepos)
#print(falsepos)
#print(falseneg)
accuracy= (truepos+trueneg)/(truepos+trueneg+falsepos+falseneg)
print(accuracy)


Accuracy =  0.7352
0.7352


In [58]:
'''
Accuracy =  0.7352
0.7352
'''

'\n0.7282\n\n'

# *LSTM* based Classifier

Use the above train and test splits.

In [230]:
# Hyperparameters of the model
vocab_size = V # choose based on statistics
oov_tok = '<OOK>'
embedding_dim = 100
max_length = 270 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

In [240]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [234]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 270, 100)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 24)                3096      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
'''
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
embedding (Embedding)        (None, 270, 100)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 24)                3096      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
=================================================================
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
'''

In [237]:
from nltk.corpus import stopwords
stopwords.words('english')
num_epochs = 5
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
1125/1125 [==============================] - 1310s 1s/step - loss: 0.3989 - accuracy: 0.8247 - val_accuracy: 0.8763 - val_loss: 0.3244
Epoch 2/5
1125/1125 [==============================] - 1316s 1s/step - loss: 0.3115 - accuracy: 0.8735 - val_accuracy: 0.8832 - val_loss: 0.3064
Epoch 3/5
1125/1125 [==============================] - 1353s 1s/step - loss: 0.2765 - accuracy: 0.8914 - val_accuracy: 0.8695 - val_loss: 0.3205
Epoch 4/5
1125/1125 [==============================] - 1348s 1s/step - loss: 0.2308 - accuracy: 0.9098 - val_accuracy: 0.8550 - val_loss: 0.3346
Epoch 5/5
1125/1125 [==============================] - 1349s 1s/step - loss: 0.2164 - accuracy: 0.9169 - val_accuracy: 0.8815 - val_loss: 0.3045


In [ ]:
'''
Epoch 1/5
1125/1125 [==============================] - 1310s 1s/step - loss: 0.3989 - accuracy: 0.8247 - val_accuracy: 0.8763 - val_loss: 0.3244
Epoch 2/5
1125/1125 [==============================] - 1316s 1s/step - loss: 0.3115 - accuracy: 0.8735 - val_accuracy: 0.8832 - val_loss: 0.3064
Epoch 3/5
1125/1125 [==============================] - 1353s 1s/step - loss: 0.2765 - accuracy: 0.8914 - val_accuracy: 0.8695 - val_loss: 0.3205
Epoch 4/5
1125/1125 [==============================] - 1348s 1s/step - loss: 0.2308 - accuracy: 0.9098 - val_accuracy: 0.8550 - val_loss: 0.3346
Epoch 5/5
1125/1125 [==============================] - ETA: 0s - loss: 0.2164 - accuracy: 0.9169
'''

In [241]:
# Calculate accuracy on Test data
'''
prediction = model.predict(test_padded)

'''
# Get probabilities
prediction = model.predict(test_padded)

# Get labels based on probability 1 if p>= 0.5 else 0
labels = list(map(lambda x:1 if x>=0.5 else 0,prediction))

# Accuracy : one can use classification_report from sklearn
print(classification_report(y_test,labels,target_names=["negative","positive"]))

              precision    recall  f1-score   support

    negative       0.90      0.85      0.88      5000
    positive       0.86      0.90      0.88      5000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [ ]:
'''
          precision    recall  f1-score   support

    negative       0.90      0.85      0.88      5000
    positive       0.86      0.90      0.88      5000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000

'''

## Get predictions for random examples

In [244]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences,padding='post',truncating='post',maxlen=max_length)




# Get probabilities
print(model.predict(padded))

# Get labels based on probability 1 if p>= 0.5 else 0

print(list(map(lambda x:"Positive" if x>=0.5 else "Negative",model.predict(padded))))



[[0.565399  ]
 [0.6092042 ]
 [0.47778553]]
['Positive', 'Positive', 'Negative']


In [ ]:

'''
[[0.565399  ]
 [0.6092042 ]
 [0.47778553]]
['Positive', 'Positive', 'Negative']

'''